In [9]:
!pip install cryptography

# 1. Symmetric Example
Simple cryptography: we generate a key, and this key is used both to encrypt and decrypt messages.
If the key is exposed, all messages encrypted with this key are exposed.

In [11]:
from cryptography.fernet import Fernet
print("Symmetric Key example")
print("first generate a key with Fernet.generate_key():")
key = Fernet.generate_key()
print(f'key: {key}')
print("Now generate a function to encrypt or decrypt stuff from using the key. func = Fernet(key)")
func = Fernet(key)
print("(function generated)")
msg_to_encrypt = input("Now give me a msg to encrypt: ")
print("to create an encrypted message we need the string to be in utf8 bytes: ")
msg_in_bytes = msg_to_encrypt.encode("utf8")
print(msg_in_bytes)
print("To encrypt we use our function. func.encrypt(msg_in_bytes). Resulting encoded message: ")
encoded_message = func.encrypt(msg_in_bytes)
print(encoded_message)
print("Now to decrypt we use our function again. func.decrypt(encoded_message)")
decoded_message = func.decrypt(encoded_message)
print("(Note that in practice, the recipient would recreate the function using the shared key)")
print("Decoded message: ")
print(decoded_message)

Symmetric Key example
first generate a key with Fernet.generate_key():
key: b'hLNlqUN-bFzKomWeR_W_mmj-6TJytklorA1nk31BNb0='
Now generate a function to encrypt or decrypt stuff from using the key. func = Fernet(key)
(function generated)


Now give me a msg to encrypt:  please encrypt me


to create an encrypted message we need the string to be in utf8 bytes: 
b'please encrypt me'
To encrypt we use our function. func.encrypt(msg_in_bytes). Resulting encoded message: 
b'gAAAAABowzS3VErBJt_l7El6SmwoF4hDdTFzQdz2RcXxx8cHALpdJQ4Hw0dOz4UrBYHeZRayK48h646QjjOqoejsYkgQvj0am_myWfAVPM9DpWnK3g0UDwk='
Now to decrypt we use our function again. func.decrypt(encoded_message)
(Note that in practice, the recipient would recreate the function using the shared key)
Decoded message: 
b'please encrypt me'


In [7]:
from cryptography.fernet import MultiFernet #having already imported Fernet above.
print("With MultiFernet, you can rotate encryption tokens.")
print("First generate 2 keys and their functions using fernet. func = ((Fernet.generate_key()) twice to get f1, f2")
key1 = Fernet.generate_key()
key2 = Fernet.generate_key()
func1 = Fernet(key1)
func2 = Fernet(key2)
print("Then create a multifernet function mfunc1 using MultiFernet([f1, f2])")
mfunc1 = MultiFernet([func1, func2])
print("Now you may encrypt a message using two keys: ")
msg_to_encode = input("Msg to encode: ")
msg_in_bytes = msg_to_encode.encode("utf8")
encrypted_message = mfunc1.encrypt(msg_in_bytes)
print(f'Encrypted message: {encrypted_message}')
decrypted_message = mfunc1.decrypt(encrypted_message)
print(f'Decrypting with our new mfunc1 we get: mfunc1.decrypt(encrypted_message) gives {decrypted_message}')
print("To rotate an encrypted message, first generate a third password, key3, and a third function, f3.")
key3 = Fernet.generate_key()
func3 = Fernet(key3)
print("Then generate the new encryption function using MultiFernet([func3, func1, func2]) to get mfunc2.\n The order matters.")
mfunc2 = MultiFernet([func3, func1, func2])
print("now re-encrypt the encrypted message using the rotate function: mfunc2.rotate(encrypted_message)")
rotated_message = mfunc2.rotate(encrypted_message)
print(f'new rotated message: {rotated_message}')
print("We can now decrypt with mfunc2.decrypt(rotated_message)")
decrypted_rotated = mfunc2.decrypt(rotated_message)
print(f'decrypted message: {decrypted_rotated}')

With MultiFernet, you can rotate encryption tokens.
First generate 2 keys and their functions using fernet. func = ((Fernet.generate_key()) twice to get f1, f2
Then create a multifernet function mfunc1 using MultiFernet([f1, f2])
Now you may encrypt a message using two keys: 


Msg to encode:  please encrypt me too


Encrypted message: b'gAAAAABowyJMrTA23_zY6FZwiYw86eTwktzOUNtadYLGFdY9tCtwqV1vplDVbPL5VA9Gmgd5_Wi7C1jMuOTD-Q8IF1RguoeY7SNnVLHGCcwDl4g3eaQAHxM='
Decrypting with our new mfunc1 we get: mfunc1.decrypt(encrypted_message) gives b'please encrypt me too'
To rotate an encrypted message, first generate a third password, key3, and a third function, f3.
Then generate the new encryption function using MultiFernet([func3, func1, func2]) to get mfunc2.
 The order matters.
now re-encrypt the encrypted message using the rotate function: mfunc2.rotate(encrypted_message)
new rotated message: b'gAAAAABowyJMvbOjqDo-ElxKxyCQ8e7TnAFac0yATKWZBefK88D7OpZ0kSmOUMFlJfdjpE4tNKwICL676qbU36qnIrO_2s_zKfWl1AqZBzxsUZidIUwRpSw='
We can now decrypt with mfunc2.decrypt(rotated_message)
decrypted message: b'please encrypt me too'


# some questions about rotation
**q: do I need to use at least two keys to create a rotatable token(encrypted message) with multifernet? why does the example start with encrypting with two keys? Is this considered best practices?**

**a**: The multifermet class expects a vector of Fermets, but it's ambiguous to me whether two 
keys are actually required at this time; best practice is to follow documented example, and
generate two keys, until one of us (the writer or reader) knows better.

# Password example: 

In [15]:
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.kdf.pbkdf2 import PBKDF2HMAC
import os, base64
print("Provide a password, which we'll encode as utf8/binary via password.encode('utf8').")
password = input("Please provide a password: ").encode("utf8")
salt = os.urandom(16)
print(f"Generate a salt: {salt} via os.random(16)")
print("Now create a kdf object using our encoder, in this case PBKDF2HMAC")
kdf = PBKDF2HMAC(
    algorithm=hashes.SHA256(),
    length=32,
    salt=salt,
    iterations=1_200_000,
)
print("""this was created via the following syntax: 
kdf = PBKDF2HMAC(
    algorithm=hashes.SHA256(),
    length=32,
    salt=salt,
    iterations=1_200_000,
)
""")
print("Now encode a key using base64.urlsafe_b64encode(kdf.derive(password))")
key = base64.urlsafe_b64encode(kdf.derive(password))
print("generate a function as usual: func = Fernet(key)")
func = Fernet(key)
print(f"Key from base64.urlsafe_b64encode(kdf.derive(password)): {key}")
secret_message = input("Please provide a message to encode: ").encode("utf8")
token = func.encrypt(secret_message)
print(f"resulting encrypted message: {token}")
decrypted_message = func.decrypt(token)
print(f"now decrypt as usual: func.decrypt(token) gives {decrypted_message}")


Provide a password, which we'll encode as utf8/binary via password.encode('utf8').


Please provide a password:  my super awesome passphrase


Generate a salt: b'\x04\xc4\xd2\xf2!\x8bp)\xc5{6\xc1\x88\x9f\xc6)' via os.random(16)
Now create a kdf object using our encoder, in this case PBKDF2HMAC
this was created via the following syntax: 
kdf = PBKDF2HMAC(
    algorithm=hashes.SHA256(),
    length=32,
    salt=salt,
    iterations=1_200_000,
)

Now encode a key using base64.urlsafe_b64encode(kdf.derive(password))
generate a function as usual: func = Fernet(key)
Key from base64.urlsafe_b64encode(kdf.derive(password)): b'7vyQTLpIaCAJUJEXcKo5S96_aHDbJHLiE8g-2t_jGpo='


Please provide a message to encode:  please encrypt this with a password (passphrase)


resulting encrypted message: b'gAAAAABowzWX6HLnWiJR9hi5JHUcEFezjxpemQjD_b2YtqCFNXtUOhBc5pakvKs6qjMclkbvATrrPG4pYcrmWYSlQzxmnzMeJbmq_xhuNPkAHM-pN6KSKit9FwCs-LtrfGOjYpooj64ps4kG52BfCWVLOQorKUHAKA=='
now decrypt as usual: func.decrypt(token) gives b'please encrypt this with a password (passphrase)'


# Elliptic key
fernet supports: 
"AES in CBC mode with a 128-bit key for encryption; using PKCS7 padding.

HMAC using SHA256 for authentication.

Initialization vectors are generated using os.urandom().
Fernet is ideal for encrypting data that easily fits in memory. As a design feature it does not expose unauthenticated bytes. This means that the complete message contents must be available in memory, making Fernet generally unsuitable for very large files at this time."
--from cryptography.io

question: AES vs SHA256? 
AES: AES is a symmetric-key encryption algorithm used for securing data at rest and in transit. It’s primarily used for encrypting and decrypting data to maintain its confidentiality.
SHA-256: SHA-256 is a cryptographic hash function used for data integrity and digital signatures. It takes an input (message) and produces a fixed-size hash value, typically 256 bits long. Its primary purpose is to verify the integrity of data and create a unique representation of a message.
Both are important to use, but for different purposes. 

question: how often to auth? when to auth? how to use HMAC and AES together? 
a: TBD

# Stream Ciphers
cha-cha example:

In [4]:
import os
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.kdf.pbkdf2 import PBKDF2HMAC

plaintext = b"This is a secret message."
key = os.urandom(32)  # ChaCha20 key must be 32 bytes
nonce = os.urandom(16)  # ChaCha20 nonce must be 16 bytes

# Create the ChaCha20 cipher object for encryption
cipher = Cipher(algorithms.ChaCha20(key, nonce), mode=None)
encryptor = cipher.encryptor()

# Encrypt the plaintext
ciphertext = encryptor.update(plaintext)
print(f"Ciphertext: {ciphertext.hex()}")

# Create the ChaCha20 cipher object for decryption
cipher = Cipher(algorithms.ChaCha20(key, nonce), mode=None)
decryptor = cipher.decryptor()

# Decrypt the ciphertext
decrypted_plaintext = decryptor.update(ciphertext)
print(f"Decrypted plaintext: {decrypted_plaintext.decode('utf-8')}")

plaintext2 = b" This is a second secret message."
ciphertext2 = encryptor.update(plaintext2)
print(f"Ciphertext2: {ciphertext2.hex()}")

decrypted_plaintext2 = decryptor.update(ciphertext2)
print(f"decrypted plaintext2: {decrypted_plaintext2.decode('utf-8')}")


Ciphertext: 10df54625d02fb10958c0f9c2ceffdce3c34fded4a874cc688
Decrypted plaintext: This is a secret message.
Ciphertext2: 85bbcfb92d532d20997333bcf52a620519a81ee5ec57b07120c8e214f9a99d9762
decrypted plaintext2:  This is a second secret message.


question: kdf.derive vs bcrypt?  bcrypt is better
but eh
kdf.derive is really good, so it hardly matters... ? 
